<a href="https://colab.research.google.com/github/mahenoor22/skin-disease-build/blob/main/model_build/VGG_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import random
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.applications.resnet50 import ResNet50

# **CallBack**

In [2]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>0.70):
            print("\nReached 70.0% accuracy so cancelling training!")
            self.model.stop_training = True

## **Import Dataset**

In [3]:
!git clone https://github.com/mahenoor22/skin-disease-build.git

fatal: destination path 'skin-disease-build' already exists and is not an empty directory.


In [4]:
train_dir = r"/content/skin-disease-build/dataset/train"
test_dir=r"/content/skin-disease-build/dataset/test"

In [5]:
assert len(os.listdir(train_dir)) == 23 and len(os.listdir(test_dir)) == 23

# **Image Generator**

In [6]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,   # randomly flip images
        validation_split = 0.2)

train_gen = datagen.flow_from_directory(train_dir,
                                       batch_size=32,
                                       class_mode='sparse',
                                       target_size=(200,200),
                                       subset = 'training')
valid_gen = datagen.flow_from_directory(train_dir,
                                       batch_size=32,
                                       class_mode='sparse',
                                       target_size=(200,200),
                                       subset = 'validation')


Found 12453 images belonging to 23 classes.
Found 3104 images belonging to 23 classes.


## Model

In [7]:
base_model = VGG16(include_top = False, input_shape = (200,200,3),pooling='avg')
model=Sequential()
model.add(base_model)
model.add(Dense(128,activation=('relu')))
model.add(Dropout(0.5))
model.add(Dense(23,activation=('softmax'))) 


for layer in base_model.layers:
    layer.trainable = False


optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(optimizer = optimizer , loss = "sparse_categorical_crossentropy", metrics=["sparse_categorical_accuracy"])
epochs = 40
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 128)               65664     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 23)                2967      
Total params: 14,783,319
Trainable params: 68,631
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
history = model.fit_generator(train_gen,epochs = epochs,
                             validation_data = valid_gen,
                             steps_per_epoch = train_gen.n//train_gen.batch_size,
                             validation_steps = valid_gen.n//valid_gen.batch_size)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
246/389 [=================>............] - ETA: 31:01 - loss: 4.4198 - sparse_categorical_accuracy: 0.1019

## **GRAPH**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()